<a href="https://colab.research.google.com/github/Sudhandra/Sudhandra_PhD/blob/main/Learning_anxiety.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#package
#!pip3 install PyTorch
!pip3 install detecto
#libraries
import torch
import os
print(torch.cuda.is_available())
from detecto import core, utils, visualize
from detecto.visualize import show_labeled_image, plot_prediction_grid
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
 import matplotlib.pyplot as plt
import numpy as np

# Data for training and validation
expressions = ['Angry', 'Contempt', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise']
training_counts = [155, 63, 172, 119, 94, 152, 57]
validation_counts = [103, 39, 125, 64, 62, 127, 44]

# Number of groups
n_groups = len(expressions)

# Create the plot
fig, ax = plt.subplots(figsize=(12, 6))

# Bar width
bar_width = 0.3

# Index positions for groups
index = np.arange(n_groups)

# Bars for training data
bar1 = plt.bar(index, training_counts, bar_width, color='red', label='Training')

# Bars for validation data
bar2 = plt.bar(index + bar_width, validation_counts, bar_width, color='lightgreen', label='Validation')

# Add labels, title, and legend
plt.xlabel('Data Distribution of Emoface Dataset')
plt.ylabel('Number of Images')
plt.title('Training and Validation Data Distribution')
plt.xticks(index + bar_width / 2, expressions)
plt.legend()

# Show plot
plt.tight_layout()
plt.show()
def imbalance_ratio(counts):
    max_count = max(counts)
    min_count = min(counts)
    return max_count / min_count

# Calculate imbalance ratio for training and validation data
training_ratio = imbalance_ratio(training_counts)
validation_ratio = imbalance_ratio(validation_counts)

print(f'Training Imbalance Ratio: {training_ratio:.2f}')
print(f'Validation Imbalance Ratio: {validation_ratio:.2f}')

In [ ]:
#dataset path
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir(r'/content/gdrive/My Drive/Emotion Intelligence/Model_20')
!ls '/content/gdrive/My Drive/Emotion Intelligence/Model_20'


In [ ]:
#image customization
custom_transforms = transforms.Compose([
transforms.ToPILImage(),
transforms.Resize(900),
transforms.RandomHorizontalFlip(0.5),
transforms.ColorJitter(saturation=0.2),
transforms.ToTensor(),
utils.normalize_transform(),])
model = core.Model.load('model_weights.pth', ['Anger','Contempt','Disgust','Fear','Happy','Sad','Surprise'])

In [ ]:
#image Prediction
image = utils.read_image('image3.jpeg')
predictions = model.predict(image)
labels, boxes, scores = predictions
show_labeled_image(image, boxes, labels)

In [ ]:
#Pruning
thresh=0.4
filtered_indices=np.where(scores<thresh)
filtered_scores=scores[filtered_indices]
filtered_boxes=boxes[filtered_indices]
num_list = filtered_indices[0].tolist()
filtered_labels = [labels[i] for i in num_list]
show_labeled_image(image, filtered_boxes, filtered_labels)


In [ ]:
labels, boxes, scores = predictions
labels,scores = np.array(labels),np.array(scores)
labels = [labels[i] for i in range(len(labels))] #if scores[i]>0.5]
labels = np.array(labels)
labels

labels,scores = np.array(labels),np.array(scores)
scores = [scores[i] for i in range(len(labels))]# if scores[i]>0.5]
scores = np.array(scores)
print(labels, scores)
type(labels)

emot_high_score= {}
emot_values = {}

for i in range(len(labels)):
    if labels[i] in emot_values:
      # Replace with the higher value between the current highest mark and the new mark
      emot_values[labels[i]] = max(emot_values[labels[i]], scores[i])
    else:
      # Keep the current highest mark if the new mark is lower
      emot_values[labels[i]] = scores[i]


# Print the resulting dictionary with the highest marks
print("Highest marks after replacement:", emot_values)


In [ ]:
P_disgust = emot_values['Disgust']
P_happy = emot_values['Happy']
P_anger = emot_values['Anger']
P_contempt = emot_values['Contempt']
P_sad = emot_values['Sad']
P_fear = emot_values['Fear']
P_surprise = emot_values['Surprise']


print(P_disgust,P_happy, P_anger, P_contempt, P_sad, P_fear, P_surprise)


w_disgust = P_disgust / sum(emot_values.values())
w_happy = P_happy / sum(emot_values.values())
w_anger = P_anger / sum(emot_values.values())
w_contempt = P_contempt / sum(emot_values.values())
w_sad = P_sad / sum(emot_values.values())
w_fear = P_fear / sum(emot_values.values())
w_surprise = P_surprise / sum(emot_values.values())
print(w_disgust, w_anger, w_contempt, w_fear, w_happy, w_sad, w_surprise )

#Learning_Anxiety = P_disgust*w_disgust +  P_fear*w_fear +  P_happy*w_happy + P_sad*w_sad + P_anger*w_anger + P_contempt*w_contempt + P_surprise*w_surprise

Learning_Anxiety = 0.56
show_labeled_image(image, filtered_boxes,  Learning_Anxiety )



In [ ]:
import matplotlib.pyplot as plt

# Sample data
#learning_anxiety_values = [0.2, 0.6, 0.8, 0.3, 0.5]
#emotion_labels = ['disgust','happy', 'anger', 'contempt', 'sad', 'fear', 'surprise')

emotion_labels = ['Disgust' ,'Happy', 'Anger', 'Contempt', 'Disgust' ,'Sad', 'Fear' ,'Surprise', 'Anger']
learning_anxiety_values = [0.8499831 , 0.4273269 , 0.41279644, 0.2504837 , 0.08079054, 0.07686906 , 0.07087852, 0.0635119,  0.05071995]
#emot_values = {}

# Image path
image_path = 'image3.jpeg'

# Load and display the image
image = plt.imread(image_path)
plt.imshow(image)
plt.axis('off')

# Overlay scatter plot
plt.scatter([100, 200, 300, 400, 500,600,700,800,900], [100, 200, 300, 400, 500,600,700,800,900], c=learning_anxiety_values, cmap='cool', s=100, alpha=0.5)

# Annotate with emotion labels
for i, txt in enumerate(emotion_labels):
    plt.annotate(txt, (100 + i*100, 100 + i*100), fontsize=12, color='black', ha='center', va='center')

# Add color bar legend
plt.colorbar(label='Learning Anxiety Value')

# Show plot
plt.show()


In [ ]:
#Learning Anxiety
if Learning_Anxiety < 0.75 and Learning_Anxiety >= 0.25:
  print('Low Anxiety')
elif Learning_Anxiety < 0.50 and Learning_Anxiety <=0.75:
  print('Mild Anxiety')
elif Learning_Anxiety == 0.50:
    print('Moderate Anxiety')
elif Learning_Anxiety < 0.50 and Learning_Anxiety > 0.25:
  print('High Anxiety')
else:
  print('Debilitating Anxiety ')